In [ ]:
# 📦 Import required libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pickle

# 🧪 Create dummy training dataset
data = {
    'temp': [25, 30, 35, 20, 22, 29, 31, 33, 24, 27],
    'hum': [60, 50, 40, 70, 80, 55, 45, 48, 72, 67],
    'moisture': [300, 400, 500, 200, 150, 450, 680, 490, 250, 260],
    'ph': [6.5, 6.8, 6.3, 5.9, 7.1, 6.0, 6.7, 6.4, 6.2, 6.9],
    'pump': [0, 1, 1, 0, 0, 1, 1, 1, 0, 0]
}

# 🧮 Create DataFrame
df = pd.DataFrame(data)

# 🧠 Separate features and target
X = df[['temp', 'hum', 'moisture', 'ph']]
y = df['pump']

# ✅ Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

# 💾 Save model to file
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ Model trained and saved as model.pkl")


In [ ]:
import serial
import time
import pickle
import pandas as pd

model = pickle.load(open("model.pkl", "rb"))
ser = serial.Serial('COM8', 9600)
time.sleep(2)

print("📡 Listening for data from NodeMCU...\n")

while True:
    try:
        raw_line = ser.readline()
        try:
            line = raw_line.decode().strip()
        except UnicodeDecodeError:
            print("⚠️ Skipped corrupted line")
            continue

        print(f"Received from NodeMCU: {line}")
        temp, hum, moisture, ph = map(float, line.split(','))

        # Proper DataFrame with feature names
        X = pd.DataFrame([[temp, hum, moisture, ph]], columns=['temp', 'hum', 'moisture', 'ph'])
        prediction = int(model.predict(X)[0])

        print("Predicted Pump Status:", prediction)
        ser.write(f"{prediction}\n".encode())
        print("Sent to NodeMCU:", prediction, "\n")

    except Exception as e:
        print("❌ General Error:", e)
